In [1]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
!pip install pandas --break-system-packages -q
!pip install pypdf --break-system-packages -q
!pip install langchain --break-system-packages -q
!pip install langchain-community --break-system-packages -q
!pip install langchain-experimental --break-system-packages -q
!pip install langchain-huggingface --break-system-packages -q
!pip install chromadb --break-system-packages -q

# Step 1 - Importing The Libraries

In [2]:
# Basic Data Processing
import pandas as pd
import json
import re

# PDF Processing
from PyPDF2 import PdfReader

# LangChain Components
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Step 1 - PDF Processing and JSON Extraction

In [3]:
def process_pdf_to_json(pdf_path, output_json_path):
    """
    Process PDF file and extract content into JSON format
    """
    # Read PDF
    reader = PdfReader(pdf_path)
    
    # Initialize list to store all documents
    documents = []
    
    # Process each page
    for page_num, page in enumerate(reader.pages):
        text = page.extract_text()
        
        # Split text into sections (you might need to adjust this based on your PDF structure)
        sections = re.split(r'\n\s*\n', text)
        
        for section in sections:
            if len(section.strip()) > 50:  # Minimum content length
                # Create document structure
                doc = {
                    "Title": f"Page {page_num + 1} - Section",  
                    "Context": section.strip(),
                    "Page": page_num + 1
                }
                documents.append(doc)
    
    # Save to JSON
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=2)
    
    print(f"Processed {len(reader.pages)} pages into {len(documents)} documents")
    return documents

In [4]:
# Process the PDF and save to JSON
pdf_path = "inputs/pdf/data.pdf"
json_output_path = "outputs/data/processed/data.json"

documents = process_pdf_to_json(pdf_path, json_output_path)

Processed 55 pages into 112 documents


# Step 2 - Loading and Restructuring JSON Data

In [5]:
def load_the_data(file_path):
    """Load and process the JSON file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
        print(f"Successfully loaded {len(data)} documents")
        return data
        
    except FileNotFoundError:
        print(f"File {file_path} not found!")
        return []
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

def analyze_the_data(data):
    """Analyze the loaded data"""
    if not data:
        return
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("\nData Analysis:")
    print(f"Total documents: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Calculate text lengths
    df['title_length'] = df['Title'].apply(len)
    df['context_length'] = df['Context'].apply(len)
    
    print(f"\nText Length Statistics:")
    print(f"Average title length: {df['title_length'].mean():.1f} characters")
    print(f"Average context length: {df['context_length'].mean():.1f} characters")
    print(f"Total context characters: {df['context_length'].sum():,}")
    
    return df

def display_documents(data, max_display=5):
    """Display the documents in a readable format"""
    print(f"\nDocuments (showing first {max_display}):")
    print("-" * 50)
    
    for i, doc in enumerate(data[:max_display]):
        print(f"\n{i+1}. {doc['Title']}")
        print(f"   {doc['Context'][:100]}...")
        print(f"   Length: {len(doc['Context'])} characters")
        print("-" * 30)

In [6]:
file_path = "outputs/data/processed/data.json"
    
# Load the data
whole_data = load_the_data(file_path)

Successfully loaded 112 documents


In [7]:
if whole_data:    
    # Display documents
    display_documents(whole_data)

    # Show Some Analysis
    analyze_the_data(whole_data)


Documents (showing first 5):
--------------------------------------------------

1. Page 1 - Section
   الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية...
   Length: 56 characters
------------------------------

2. Page 2 - Section
   2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة   
عدد السكان1 : ...
   Length: 1898 characters
------------------------------

3. Page 2 - Section
   1 - الجهاز المركزي للتعبئة العامة واالحصاء؛ الكتاب  االحصائي السنوي، سبتمبر 2015. 
2 - الجهاز المركز...
   Length: 205 characters
------------------------------

4. Page 3 - Section
   3 
  الناتج المحلىاإلجمالي4  المصري عن عام 2013/2014  = بسعر السوق 1643,4 .مليار جنية مصري 5 
 معدل ...
   Length: 180 characters
------------------------------

5. Page 3 - Section
   4  الناتج المحلى االجماليبسعر السوق: ما انتجه من سلع وخدمات باألسعار الجارية مستبعد فئة المستلزمات ا...
   Length: 428 characters
------------------------------

Data Analysis:
Total documents: 112
C

# Step 3 - Document Creation

In [8]:
def Create_documents(whole_data):
    """
    Create documents suitable for embedding models
    Returns: List of Document objects with combined text for embedding
    """
    documents = []
    
    for item in whole_data:
        if isinstance(item, list):
            item = item[0]

        title = item.get("Title", "")
        context = item.get("Context", "")

        if title and context:
            # Combine title and context for better embedding quality
            combined_text = f"{title}\n\n{context}"
            
            documents.append(Document(
                page_content=combined_text,  # This is what gets embedded
                metadata={
                    "title": title,
                    "context": context,
                    "text_length": len(combined_text)
                }
            ))
    
    return documents

In [9]:
# Usage:
documents = Create_documents(whole_data)

In [10]:
print(f"Documents lenght : {len(documents)}")

for i, doc in enumerate(documents[:2]):
    print(f"\nDocument {i+1}:")
    print(f"Title : {doc.page_content[:100]}...")
    print(f"Meta Data : {doc.metadata}")
    print("-" * 50)

Documents lenght : 112

Document 1:
Title : Page 1 - Section

الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية...
Meta Data : {'title': 'Page 1 - Section', 'context': 'الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية', 'text_length': 74}
--------------------------------------------------

Document 2:
Title : Page 2 - Section

2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة...
Meta Data : {'title': 'Page 2 - Section', 'context': '2 \n  النظام القانوينجلهموورية صصر العريية   \nالسياق العام للبلد \n العاصمة: القاهرة   \nعدد السكان1 : 96,521,618 \nاللغة الرسمية: العربية \nالمساحة الجغرافية: تقع جمهورية مصر العربية فى الشمال الشرقي لقار ة \nإفريقيا، وتطل على كل من الساحل الجنوبي الشرقي للبحر المتوسط \nوالساحل الشمالي الغربي للبحر االحمر بمساحة اجمالية تبلغ مليون كم 2 \n تقريباً، مصر دولة إفريقية غير أن جزءا من أراضيها، وهى شبه جزيرة\nسيناء" يقع في قارة أسيا. \nلمصر حدود مشتركة مع ليبيا من الغرب، مع السودان من الجنوب، ومع \nاسرائيل وفلسطين (

In [11]:
# For embeddings, you'll use:
texts = [doc.page_content for doc in documents] 

print(f"Created {len(documents)} documents for embedding")
print(f"Sample text for embedding: {texts[1][:]}...")

Created 112 documents for embedding
Sample text for embedding: Page 2 - Section

2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة   
عدد السكان1 : 96,521,618 
اللغة الرسمية: العربية 
المساحة الجغرافية: تقع جمهورية مصر العربية فى الشمال الشرقي لقار ة 
إفريقيا، وتطل على كل من الساحل الجنوبي الشرقي للبحر المتوسط 
والساحل الشمالي الغربي للبحر االحمر بمساحة اجمالية تبلغ مليون كم 2 
 تقريباً، مصر دولة إفريقية غير أن جزءا من أراضيها، وهى شبه جزيرة
سيناء" يقع في قارة أسيا. 
لمصر حدود مشتركة مع ليبيا من الغرب، مع السودان من الجنوب، ومع 
اسرائيل وفلسطين (قطاع غزة) من الشمال الشرقي، ويمر الممر المائي (قناة 
السويس) الذى يربط البحر المتوسط والبحر االحمر عبر االراضي المصرية 
فاصالً الجزء األفريقي منها وهو األكبر مساحة عن الجزء األسيوي األصغر 
منها"، تشكل الصحراء غالبية مساحة مصر، ويتركز أغلب سكان الجمهورية  
في وادى النيل والدلتا2. 
 كماجاء في دستورها الجديد( 2014 )،جمهورية مصر العربية دولة ذات سيادة، موحدة ال تقبل التجزئة، وال  
ينزل عن شيء منها، نظامها جمهوري  دي

# Step 4 - Create The Vector Database - ChromaDB

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v1")
splitter = SemanticChunker(embeddings)

2025-09-14 08:03:31.173121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [13]:
chunks = splitter.split_documents(documents)

In [16]:
chunks[20]

Document(metadata={'title': 'Page 8 - Section', 'context': '8 \n  الثالثة، والملكية العامة, والملكية الخاصة, والملكية التعاونية"40 ."للملكية العامة حرمة, ال يجوز المساس بها, \nوحمايتها واجب وفقا للقانون". 41 "السجن دار إصالح وتأهيل. تخضع السجون  وأماكن االحتجاز....ويحظر \nفيها كل ما ينافي كرامة اإلنسان, أو يعرض صحته للضرر".42 "الكرامة حق لكل إنسان, وال يجوز المساس \nبها, وتلتزم الدولة باحترامها وحمايتها".43 "كل من يقبض عليه, أو يحبس, أو تقيد حريته تجب معاملته بما \nيحفظ عليه كرامته, وال يجوز تعذيبه, وال ترهيبه, وال إكراهه, وال إيذاؤه بدنيا أو معنويا"44  ."السجن دار \nإصالح وتأهيل... ويحظر فيها كل ما ينافي كرامة اإلنسان, أو يعرض صحته للضرر...." 45 "للحياة \nالخاصة حرمة, وهي مصونة ال تمس. وللمراسالت البريدية, والبرقية, واإللكترونية, والمحادثات الهاتفية, \nوغيرها من وسائل االتصال حرمة, وسريتها مكفولة, وال تجوز مصادرتها, أو االطالع عليها, أو رقابتها .... \nكما تلتزم الدولة بحماية حق المواطنين في استخدام وسائل االتصال العامة بكافة أشكالها, وال يجوز تعطيلها \nأو وقفها أو حرمان المواطنين منها

Document(metadata={'title': 'Page 8 - Section', 'context': '8 \n  الثالثة، والملكية العامة, والملكية الخاصة, والملكية التعاونية"40 ."للملكية العامة حرمة, ال يجوز المساس بها, \nوحمايتها واجب وفقا للقانون". 41 "السجن دار إصالح وتأهيل. تخضع السجون  وأماكن االحتجاز....ويحظر \nفيها كل ما ينافي كرامة اإلنسان, أو يعرض صحته للضرر".42 "الكرامة حق لكل إنسان, وال يجوز المساس \nبها, وتلتزم الدولة باحترامها وحمايتها".43 "كل من يقبض عليه, أو يحبس, أو تقيد حريته تجب معاملته بما \nيحفظ عليه كرامته, وال يجوز تعذيبه, وال ترهيبه, وال إكراهه, وال إيذاؤه بدنيا أو معنويا"44  ."السجن دار \nإصالح وتأهيل... ويحظر فيها كل ما ينافي كرامة اإلنسان, أو يعرض صحته للضرر...." 45 "للحياة \nالخاصة حرمة, وهي مصونة ال تمس. وللمراسالت البريدية, والبرقية, واإللكترونية, والمحادثات الهاتفية, \nوغيرها من وسائل االتصال حرمة, وسريتها مكفولة, وال تجوز مصادرتها, أو االطالع عليها, أو رقابتها .... \nكما تلتزم الدولة بحماية حق المواطنين في استخدام وسائل االتصال العامة بكافة أشكالها, وال يجوز تعطيلها \nأو وقفها أو حرمان المواطنين منها, بشكل تعسفي, ..."46. "للمنازل حرمة, وفيما عدا حاالت الخطـر, أو \nاالستغاثة ال يجوز دخولها, وال تفتيشها, وال مراقبتها أو التنصت عليها..."47 . "لجسد اإلنسان حرمة, \nواالعتداء عليه, أو تشويهه, أو التمثيل به, جريمة يعاقب عليها القانون . ويحظر االتجار بأعضائه, وال يجوز \nإجراء أية تجربة طبية, أو علمية عليه بغير رضاه الحر الموثق, .... "48 ."يحظر التهجير القسري  التعسفي \nللمواطنين بجميع صوره وأشكاله,...."49 . "المعلومات والبيانات واإلحصاءات والوثائق الرسمية ملك للشعب, \nواإلفصاح عنها من مصادرها المختلفة, حق تكفله الدولة لكل مواطن, وتلتزم الدولة بتوفيرها وا تاحتها \nللمواطنين بشفافية, ..."50 "تلتزم الدولة بحماية حقوق  الملكية الفكرية بشتي أنواعها في كافة \nالمجاالت,...."51 "يحظر بأي وجه فرض رقابة علي الصحف ووسائل اإلعالم المصرية أو مصادرتها أو \nوقفها أو إغالقها.... وال  توقع عقوبة سالبة للحرية في الجرائم التي ترتكب بطريق النشر أو العالنية, ..."52 \n"تلتزم الدولة بضمان استقالل المؤسسات الصحفية ووسائل اإلعالم المملوكة لها, بما يكفل حيادها, وتعبيرها', 'text_length': 1894}, page_content='"48 ."يحظر التهجير القسري  التعسفي \nللمواطنين بجميع صوره وأشكاله,...."49 . "المعلومات والبيانات واإلحصاءات والوثائق الرسمية ملك للشعب, \nواإلفصاح عنها من مصادرها المختلفة, حق تكفله الدولة لكل مواطن, وتلتزم الدولة بتوفيرها وا تاحتها \nللمواطنين بشفافية, ..."50 "تلتزم الدولة بحماية حقوق  الملكية الفكرية بشتي أنواعها في كافة \nالمجاالت,...."51 "يحظر بأي وجه فرض رقابة علي الصحف ووسائل اإلعالم المصرية أو مصادرتها أو \nوقفها أو إغالقها.... وال  توقع عقوبة سالبة للحرية في الجرائم التي ترتكب بطريق النشر أو العالنية, ..."52 \n"تلتزم الدولة بضمان استقالل المؤسسات الصحفية ووسائل اإلعالم المملوكة لها, بما يكفل حيادها, وتعبيرها')


In [17]:
Data_Dir = "./outputs/data/vector_database" 

vectorstore = Chroma.from_documents(
    documents=chunks,          
    embedding=embeddings,      
    persist_directory=Data_Dir  
)

vectorstore.persist()

/tmp/ipykernel_16913/2220787854.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
vectorstore1 = Chroma(
    persist_directory=Data_Dir, 
    embedding_function=embeddings
)

/tmp/ipykernel_603719/2776979415.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore1 = Chroma(


In [21]:
vectorstore_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

documents = vectorstore_retriever.get_relevant_documents("ما هي بنود اتفاقية حقوق عمل المرأة")
texts = [doc.page_content for doc in documents]
print(f"Created {len(documents)} documents for embedding")
print(f"Sample text for embedding: {texts[1][:]}...")

Created 3 documents for embedding
Sample text for embedding: كما تكفل للمرأة حقها في تولى الوظائف العامة ووظائف اإلدارة العليا  في الدولة والتعيين في 
الجهات والهيئات القضائية، دون تمييز ضدها  ... وتكفل تمكين المرأة من التوفيق بين واجبات األسرة 
ومتطلبات العمل. كما تلتزم بتوفير الرعاية والحماية لألمومة والطفولة والمرأة المعيلة والمسنة والنساء 
األشد احتياجا. "295   
و"العمل حق وواجب، وشرف تكفله الدولة. وال يجوز إلزام  أي مواطن بالعمل جبراً، إال بمقتضى قانون، 
وألداء خدمة عامة، لمدة محددة، وبمقابل عادل، ودون إخالل بالحقوق األساسية للمكلفين بالعمل. " 296  
و"تلتزم الدولة بالحفاظ على حقوق العمال، وتعمل على بناء عالقا ت عمل متوازنة بين طر في العملية 
اإلنتاجية، وتكفل سبل التفاوض الجماعي، وتعمل على حماية العمال من مخاطر العمل وتوافر شروط  
األمن والسالمة والصحة المهنية، ويحظر فصلهم تعسفياً، وذلك كله على النحو الذى ينظمه 
القانون ."297  
"والوظائف العامة حق للمواطنين على أساس الكف اءة، ودون محاباة أو وساطة، وتكليف للقائمين بها 
لخدمة الشعب، وتكفل الدولة حقوقهم وحمايتهم، وقيامهم بأداء واجباته